In [1]:
import pandas as pd
import numpy as np
import pylab as pl

In [3]:
df = pd.read_csv("credit-data-post-import.csv")

In [4]:
is_test = np.random.uniform(0, 1, len(df)) > 0.75
train = df[is_test==False]
test = df[is_test==True]

In [35]:
len(train), len(test)

(112415, 37585)

In [6]:
from sklearn.neighbors import KNeighborsRegressor

income_imputer = KNeighborsRegressor(n_neighbors=1)

#split our data into 2 groups; data containing nulls and data 
# not containing nulls we'll train on the latter and make
# 'predictions' on the null data to impute monthly_income
train_w_monthly_income = train[train.monthly_income.isnull()==False]
train_w_null_monthly_income = train[train.monthly_income.isnull()==True]

In [7]:
train_w_monthly_income.corr()

,serious_dlqin2yrs,revolving_utilization_of_unsecured_lines,age,number_of_time30-59_days_past_due_not_worse,debt_ratio,monthly_income,number_of_open_credit_lines_and_loans,number_of_times90_days_late,number_real_estate_loans_or_lines,number_of_time60-89_days_past_due_not_worse,number_of_dependents
serious_dlqin2yrs,1.000000,-0.004287,-0.103414,0.122708,-0.003785,-0.018172,-0.025160,0.110690,-0.003111,0.093507,0.046192
revolving_utilization_of_unsecured_lines,-0.004287,1.000000,-0.006662,-0.001046,0.000141,0.007737,-0.010511,-0.000919,0.007436,-0.001032,0.003657
age,-0.103414,-0.006662,1.000000,-0.047833,0.000368,0.033983,0.185555,-0.046722,0.065972,-0.042228,-0.205781
number_of_time30-59_days_past_due_not_worse,0.122708,-0.001046,-0.047833,1.000000,-0.002442,-0.008805,-0.043294,0.974676,-0.024055,0.979656,0.005095
debt_ratio,-0.003785,0.000141,0.000368,-0.002442,1.000000,-0.029130,0.008229,-0.002986,0.020682,-0.002242,0.011094
monthly_income,-0.018172,0.007737,0.033983,-0.008805,-0.029130,1.000000,0.085640,-0.011260,0.117335,-0.009766,0.058499
number_of_open_credit_lines_and_loans,-0.025160,-0.010511,0.185555,-0.043294,0.008229,0.085640,1.000000,-0.071938,0.427258,-0.060916,0.039757
number_of_times90_days_late,0.110690,-0.000919,-0.046722,0.974676,-0.002986,-0.011260,-0.071938,1.000000,-0.040298,0.988802,-0.002384
number_real_estate_loans_or_lines,-0.003111,0.007436,0.065972,-0.024055,0.020682,0.117335,0.427258,-0.040298,1.000000,-0.034097,0.120309
number_of_time60-89_days_past_due_not_worse,0.093507,-0.001032,-0.042228,0.979656,-0.002242,-0.009766,-0.060916,0.988802,-0.034097,1.000000,-0.003322


In [9]:
train_w_monthly_income.corr().ix[:,5]

G:\Installations\an3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if __name__ == '__main__':


serious_dlqin2yrs                             -0.018172
revolving_utilization_of_unsecured_lines       0.007737
age                                            0.033983
number_of_time30-59_days_past_due_not_worse   -0.008805
debt_ratio                                    -0.029130
monthly_income                                 1.000000
number_of_open_credit_lines_and_loans          0.085640
number_of_times90_days_late                   -0.011260
number_real_estate_loans_or_lines              0.117335
number_of_time60-89_days_past_due_not_worse   -0.009766
number_of_dependents                           0.058499
Name: monthly_income, dtype: float64

In [10]:
cols = ['NumberRealEstateLoansOrLines', 'NumberOfOpenCreditLinesAndLoans']
income_imputer.fit(train_w_monthly_income[cols], train_w_monthly_income.MonthlyIncome)

KeyError: "['NumberRealEstateLoansOrLines' 'NumberOfOpenCreditLinesAndLoans'] not in index"

In [40]:
new_values = income_imputer.predict(train_w_null_monthly_income[cols])


In [41]:
train_w_null_monthly_income['monthly_income'] = new_values
new_values

array([  7666.,   3235.,   9666., ...,   2155.,   3771.,  10000.])

In [42]:
#combine the data back together
train = train_w_monthly_income.append(train_w_null_monthly_income)
len(train)

112415

In [43]:
test['monthly_income_imputed'] = income_imputer.predict(test[cols])
test.head()

,serious_dlqin2yrs,revolving_utilization_of_unsecured_lines,age,number_of_time30-59_days_past_due_not_worse,debt_ratio,monthly_income,number_of_open_credit_lines_and_loans,number_of_times90_days_late,number_real_estate_loans_or_lines,number_of_time60-89_days_past_due_not_worse,number_of_dependents,monthly_income_imputed
3,0,0.233810,30,0,0.036050,3300,5,0,0,0,0,6017
13,1,0.964673,40,3,0.382965,13700,9,3,1,1,2,2850
16,0,0.061086,78,0,2058.000000,NaN,10,0,2,0,0,2500
23,0,0.075427,32,0,0.085512,7916,6,0,0,0,0,4145
24,0,0.046560,58,0,0.241622,2416,9,0,1,0,0,2850


In [44]:
test['monthly_income'] = np.where(test.monthly_income.isnull(), test.monthly_income_imputed,
                                  test.monthly_income)

In [45]:
print pd.value_counts(train.monthly_income.isnull())
print pd.value_counts(test.monthly_income.isnull())

False    112415
False    37585


In [46]:
train.to_csv("./data/credit-data-trainingset.csv", index=False)
test.to_csv("./data/credit-data-testset.csv", index=False)